1. Prepare for Neurospin2BIDS

In [ ]:
# Make participants_to_import.tsv

# This file is not part of the BIDS standard.
# It is defined to contain the minimum information needed to simplify creating a BIDS dataset with the data from the NeuroSpin server.

_to_import = []

# ---------- Anatomical Runs ----------
inv1 = ([ID OF FILE], "anat", "inv1")
inv2 = ([ID OF FILE], "anat", "inv2")
t1 = ([ID OF FILE], "anat", "t1")
uni = ([ID OF FILE], "anat", "uni")

_to_import.append(inv1)
_to_import.append(inv2)
_to_import.append(t1)
_to_import.append(uni)

# ---------- Functional Runs ----------
for tsk in ITERATE_THROUGH_TASKS:

    # BOLD
    for run_num, run_id in enumerate([ID OF FILES],1):
        bold = (run_id, "func", [PATH])
        _to_import.append(bold)

    # SBREF
    for run_num, run_id in enumerate([ID OF FILES],1):
        sbref = (run_id, "func", [PATH])
        _to_import.append(sbref)

    # FMAP
    for run_num, run_id in enumerate([ID OF FILES],1):
        fmap = (run_id, "fmap", [PATH])
        _to_import.append(fmap)

# Prepare the structure of the file to save
bids_info = {
    'participant_id': f"sub-{sid}",
    'NIP': [NIP],
    'infos_participant': json.dumps({"sex":[SEX],"age":[AGE]}),
    'session_label': '',
    'acq_date': [DATE],
    'acq_label':'',
    'location': [LOCATION_ACQUISITION],
    'to_import': _to_import,
    'func_runs_num': [NUMBER_OF_RUNS]
}

bids_info = pd.DataFrame([bids_info])

# Save the file
_file_to_create = 'participants_to_import.tsv'
bids_info.to_csv(os.path.join(_file_to_create), sep="\t", index=False)

In [ ]:
# Run runNeurospin2BIDS.sh

_args_sh = [[ROOT_FOLDER],[BIDS_FOLDER],[ACQUISITION_FOLDER]]
os.chmod('runNeurospin2BIDS.sh', stat.S_IRWXU)
result = subprocess.run('runNeurospin2BIDS.sh' + _args_sh, capture_output=True, text=True)

# Print the output and error (if any)
print("Output:")
print(result.stdout)

print("Error:")
print(result.stderr)

# Print the return code
print("Return Code:", result.returncode)

2. Denoise the anatomy file

In [ ]:
multiplying_factor = 10

MP2RAGE_filenameUNI = [PATH_TO FILE]

MP2RAGE_filenameINV1 = [PATH_TO FILE]

MP2RAGE_filenameINV2 = [PATH_TO FILE]

MP2RAGE_uniden_output_filename = [PATH_TO FILE]

mp2rage_genUniDen(MP2RAGE_filenameUNI, MP2RAGE_filenameINV1, MP2RAGE_filenameINV2, 
                    MP2RAGE_uniden_output_filename, multiplying_factor)

In [ ]:
# Change the header

print('Now changing the JSON file header...')

# Input JSON file
FILE_JSON_INPUT = [PATH_TO FILE]

# Output JSON file
FILE_JSON_OUTPUT = [PATH_TO FILE]

with open(FILE_JSON_INPUT, 'r') as file_input:
    ANAT_JSON = json.load(file_input)

# Modify the desired elements in the Python data structure
ANAT_JSON['SeriesDescription'] = 'mp2rage_iso0.65_iPAT2_angulated_UNI_Images_DEN'
ANAT_JSON['ImageComments'] = 'MP2RAGE Uni Image denoised'

# Open the output JSON file for writing
with open(FILE_JSON_OUTPUT, 'w') as file_output:
    json.dump(ANAT_JSON, file_output)

3. Run fMRIPrep

In [ ]:
# Make and update the FMAP JSON file

# ---------- Make a copy of SBREF file and save it as FMAP with AP direction ----------
# FMAP - direction: PA
FMAP_PA_LONG_PATH = [PATH_TO FILE]

# FMAP - direction: AP
FMAP_AP_LONG_PATH = [PATH_TO FILE]

# SBREF - direction: AP
SBREF_AP_LONG_PATH = [PATH_TO FILE]

# Copy file
subprocess.run(f'cp {SBREF_AP_LONG_PATH}.nii.gz {FMAP_AP_LONG_PATH}.nii.gz', shell=True)
subprocess.run(f'cp {SBREF_AP_LONG_PATH}.json {FMAP_AP_LONG_PATH}.json', shell=True)

# ---------- Update FMAP json file with AP direction ----------
BOLD_AP_SHORT_PATH = [PATH_TO FILE]

_entities = {'direction':'ap','run':f"{run_num:02}",'suffix':'sbref','task':tsk}
SBREF_AP_SHORT_PATH = [PATH_TO FILE]

# Read the JSON files
FMAP_AP_JSON = dict()
with open(f'{FMAP_AP_LONG_PATH}.json', 'r') as f:
    FMAP_AP_JSON = json.load(f)

FMAP_AP_JSON['B0FieldIdentifier'] = f'pepolar_{tsk}{run_num:02}'
FMAP_AP_JSON['IntendedFor'] = [f'{BOLD_AP_SHORT_PATH}',f'{SBREF_AP_SHORT_PATH}']

with open(f'{FMAP_AP_LONG_PATH}.json', 'w') as f:
    json.dump(FMAP_AP_JSON,f, indent=2)

# ---------- Update FMAP json file with PA direction ----------        
# Read the JSON files
FMAP_PA_JSON = dict()
with open(f'{FMAP_PA_LONG_PATH}.json', 'r') as f:
    FMAP_PA_JSON = json.load(f)

FMAP_PA_JSON['B0FieldIdentifier'] = f'pepolar_{tsk}{run_num:02}'
FMAP_PA_JSON['IntendedFor'] = [f'{BOLD_AP_SHORT_PATH}',f'{SBREF_AP_SHORT_PATH}']

with open(f'{FMAP_PA_LONG_PATH}.json', 'w') as f:
    json.dump(FMAP_PA_JSON,f, indent=2)

In [ ]:
# Ignore phase file inside the func folder

BIDS_FUNC_FOLDER = [PATH_TO_FOLDER]

for filename in os.listdir(BIDS_FUNC_FOLDER):
    if "part-phase" in filename and not filename.endswith(".ignore"):
        # Construct the full path to the file
        old_file_path = os.path.join(BIDS_FUNC_FOLDER, filename)
        
        # Create the new filename with ".ignore" appended
        new_filename = f"{filename}.ignore"
        new_file_path = os.path.join(BIDS_FUNC_FOLDER, new_filename)
        
        # Rename the file
        os.rename(old_file_path, new_file_path)
        print(f"Renamed: {old_file_path} -> {new_file_path}")

In [ ]:
# Run runfMRIprep.sh

_args_sh = _args_sh = [[ROOT_FOLDER],[BIDS_FOLDER],[DESTINATION_FOLDER],[PARTICIPANT_ID]]
os.chmod('runfMRIprep.sh', stat.S_IRWXU)
result = subprocess.run('runfMRIprep.sh' + _args_sh, capture_output=True, text=True)

# Print the output and error (if any)
print("Output:")
print(result.stdout)

print("Error:")
print(result.stderr)

# Print the return code
print("Return Code:", result.returncode)